## Clasa Node pe care o vom folosi pentru a stoca informatii despre fiecare pozitie

In [42]:
class node:
    def __init__(self, infoNod, coordX, coordY, succesori=None, parent=None):
        self.coordX = coordX
        self.coordY = coordY
        self.infoNod = infoNod
        self.parent = parent
        self.g = None  # costul de la radacina la nodul curent
        self.h = None  # costul estimat de la nodul curent la nodul tinta
        self.f = 0  # costul total (g + h)
        self.succesori = succesori

        self.expended = False

    def drumRadacina(self):
        drum = []
        nodCurent = self
        while nodCurent != None:
            drum.append(nodCurent)
            nodCurent = nodCurent.parent

        drum.reverse()

        return drum
    
    def vizitat(self):
        return len([1 for nod in self.drumRadacina() if nod == self]) > 1
    
    def __lt__(self, other):
        if self.infoNod == other.infoNod:
            return self.g > other.g
        return self.infoNod < other.infoNod
    
    def __str__(self):
        return f"Node({self.infoNod}, {self.coordX}, {self.coordY})"
    
    def __repr__(self):
        return f"{self.infoNod}"

In [ ]:
import heapq


class graph:
    def __init__(self, startNode, goalNodes, nodesData, adjList):

        #Initializam nodurile din jurul Olimpului

        self.nodes = {info : node(info, coords[0], coords[1]) for info, coords in nodesData.items()}

        for info, succ_indices in adjList.items():
            self.nodes[info].succesori = [self.nodes[i] for i in succ_indices]


        self.startNode = self.nodes[startNode]
        self.startNode.g = 0
        self.startNode.h = float('inf')
        self.goalNodes = [self.nodes[goal] for goal in goalNodes]

    def _euclidian(self, node1, node2):
        dx = abs(node1.coordX - node2.coordX)
        dy = abs(node1.coordY - node2.coordY)
        return (dx * dx + dy * dy)**0.5

    def _heuristic(self, node):
        return min(self._euclidian(node, goalNode) for goalNode in self.goalNodes)

    def AStar(self):
        openList = [self.startNode]
        closedList = []
        while openList:
            currentNode = openList.pop(0)
            currentNode.expended = True
            closedList.append(currentNode)

            if currentNode in self.goalNodes:
                print(f"gasit {currentNode.infoNod}")
                return currentNode.drumRadacina()

            for successor in currentNode.succesori:
                newG = currentNode.g + self._euclidian(currentNode, successor)
                newH = self._heuristic(successor)
                newF = newG + newH
                newNode = None
                if successor not in currentNode.drumRadacina():
                    if successor in openList:
                        for openNode in openList:
                            if openNode.f > newF or (openNode.f == newF and openNode.g < newG):
                                openList.remove(openNode)
                                heapq.heapify(openList)
                                openNode.g = newG
                                openNode.h = newH
                                openNode.f = newF
                                openNode.parent = currentNode
                    if successor.expended:
                        for closedNode in closedList:
                            if closedNode.f > newF or (closedNode.f == newF and closedNode.g < newG):
                                closedList.remove(closedNode)
                                heapq.heapify(closedList)
                                closedNode.g = newG
                                closedNode.h = newH
                                closedNode.f = newF
                                closedNode.parent = currentNode
                                newNode = successor
                    else:
                        newNode = successor
                        newNode.g = newG
                        newNode.h = newH
                        newNode.f = newF
                        newNode.parent = currentNode

                    if newNode:
                        heapq.heappush(openList, newNode)

        return None

In [44]:
nodes_data = {
    1: (0, 0),
    2: (2, 0),
    3: (4, 0),
    4: (0, 2),
    5: (2, 2),
    6: (4, 2),
    7: (1, 3),
    8: (2, 3),
    9: (3, 3),
    10: (0, 4),
    11: (1, 4),
    12: (2, 4),
    13: (3, 4),
    14: (4, 4),
    15: (5, 4),
    16: (1, 5),
    17: (2, 5),
    18: (3, 5),
    19: (0, 6),
    20: (2, 6),
    21: (4, 6),
    22: (0, 8),
    23: (2, 8),
    24: (4, 8),
}

connections = {
    1: [2, 10],
    2: [1, 3, 5],
    3: [2, 15],
    4: [5, 11],
    5: [4,2,6,8],
    6: [5,14],
    7: [8,12],
    8: [7,5,9],
    9: [8,13],
    10: [1,22,11],
    11: [10,4,12,19],
    12: [7,11,16],
    13: [9,18,14],
    14: [13,6,21,15],
    15: [3,14,24],
    16: [12,17],
    17: [16,18,20],
    18: [13,17],
    19: [11,20],
    20: [19,17,21,23],
    21: [14,20],
    22: [10,23],
    23: [22,20,24],
    24: [23,15],
}

g = graph(12, [1,20], nodes_data, connections)

print(g.AStar())

Verific Node(12, 2, 4)
openList: []
Verific Node(7, 1, 3)
openList: [11, 16]
Verific Node(8, 2, 3)
openList: [16, 11]
Verific Node(5, 2, 2)
openList: [9, 16, 11]
Verific Node(2, 2, 0)
openList: [4, 6, 16, 9, 11]
Verific Node(1, 0, 0)
openList: [6, 3, 9, 11, 16, 4]
gasit 1
[12, 7, 8, 5, 2, 1]
